In [1]:
%matplotlib tk
import pycrystem as pc
import numpy as np
import hyperspy.api as hs
from matplotlib import pyplot as plt

In [8]:
from pycrystem.indexation_generator import *
from pycrystem.utils.expt_utils import *
from pycrystem import ElectronDiffraction
from pycrystem.indexation_generator import IndexationGenerator,mapable_radial_matcher

In [3]:
%%time
folder = '/home/pc494/Documents/data/pattern_matching_in_perovskites'

dp = pc.load(folder+'QD-03.blo')
dp = dp.inav[100:110,100:110]
dp.apply_affine_transformation(D = np.array([[0.99, 0.00, 0.00],
                                             [0.00, 0.69, 0.00],
                                             [0.00, 0.00, 1.00]]),show_progressbar=False)
dp.set_calibration(0.032)

A Jupyter Widget


CPU times: user 300 ms, sys: 132 ms, total: 432 ms
Wall time: 208 ms


In [4]:
%%time
radial_averaged_data = dp.map(radial_average,center=np.asarray([144/2,144/2]),cython=False,inplace=False, show_progressbar=False)

CPU times: user 96 ms, sys: 12 ms, total: 108 ms
Wall time: 59.5 ms


In [5]:
%%time
structure = pc.Structure.from_file(folder+"CsPbBr3_1.cif")
rot_array = np.loadtxt(folder + 'mmm_grid_euler.bin')
rot_list = rot_array.tolist()[0:500] #!!!
edc = pc.ElectronDiffractionCalculator(300, 0.025)
diff_gen = pc.DiffractionLibraryGenerator(edc)
struc_lib = dict()
struc_lib["CsPbBr3"] = (structure, rot_list)

CPU times: user 368 ms, sys: 4 ms, total: 372 ms
Wall time: 373 ms


In [6]:
%%time
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=1.2/128,
                                            reciprocal_radius=1.,
                                            representation='euler')

CPU times: user 1.21 s, sys: 0 ns, total: 1.21 s
Wall time: 1.21 s


In [7]:
%%time
indexer = IndexationGenerator(dp, library)
match_results = indexer.correlate(n_largest=5)

A Jupyter Widget

CPU times: user 14.2 s, sys: 2.16 s, total: 16.4 s
Wall time: 13.8 s


In [9]:
%%time
radial_sims = {}
for XZX in library['CsPbBr3']:
    peaks_2D_data = library['CsPbBr3'][XZX].as_signal(144,0.3,2.1).data
    radial_sims[XZX] = radial_average(peaks_2D_data,center=[144/2,144/2],cython=False)

CPU times: user 1min 27s, sys: 1.78 s, total: 1min 29s
Wall time: 59.2 s


In [10]:
%%time
alpha = radial_averaged_data.map(mapable_radial_matcher,dict_of_radial_patterns=radial_sims,n_scorer=10,inplace=False)

A Jupyter Widget